# Surfrider ETL script

## Blob

In [ ]:
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient

''' blobContainer create a name_list of blobs within container'''
''' params are container name (no full url) & storage conn string'''
def blobInContainer(connection_s,container_n):
    campaign_container = ContainerClient.from_connection_string(conn_str=connection_s, container_name=container_n)
    blob_list = campaign_container.list_blobs()
    blob_names_list = []
    for blob in blob_list:
        blob_names_list.append(blob.name)
    return blob_names_list

''' blobInfos provides basic information about a blob object'''
''' params are blob_name only (no full url) & storage conn string'''
def blobInfos(connection_s,container_n,blob_n):
    blob_video = BlobClient.from_connection_string(conn_str=connection_s,container_name=container_n, blob_name=blob_n)
    blob_video_url = blob_video.url
    blob_video_prop = blob_video.get_blob_properties()
    blob_video_prop_keys = blob_video_prop.keys()
    print("blob name:",blob_n)
    print("blob URL:",blob_video_url)
    print("blob properties:", blob_video_prop)
    print("blob properties keys:", blob_video_prop_keys)


''' downloadBlob from Azure to local file system'''
''' parameter is a blob client object from azure storage sdk'''
def downloadBlob(blobclient):
    with open("/tmp/"+blobclient.blob_name, "wb") as my_blob_dl:
        blob_data = blobclient.download_blob()
        blob_data.readinto(my_blob_dl)
    print("Blob %s downloaded" %blobclient.blob_name)
    print("Blob path: /tmp/%s" %blobclient.blob_name)
    path = "/tmp/"+blobclient.blob_name
    return path

## AI Prediction

In [ ]:
import json 
import os
import subprocess
import requests

def isAIready(url):
    AI_ready = requests.get(url)
    if AI_ready.status_code == 200:
        print('AI is ready')
        print('HTTP Status code: ',AI_ready.status_code)
        print(AI_ready.headers)
    else:
        print("AI not found, an error has occured")
        print("Server Answer: ",AI_ready)
        print("HTTP Status Code: ",AI_ready.status_code)
        print("HTTP Status Code: ",AI_ready.raise_for_status())

''' getPrediction function sends curl request to Surfrider AI'''
''' video name is the name of a locally downloaded video from Azure'''
''' video name is passed as an argument to curl_request script '''
''' curl_request script sends actual POST request to Surfrider AI'''
def getPrediction(video_name):
    curl_request_script = ['./curl_request_param.sh',video_name]
    output = []
    request_answer = subprocess.Popen(curl_request_script, stdout=subprocess.PIPE)
    i = 0
    for line in request_answer.stdout:
        print(line)
        output.append(line)
    return output


''' jsonPrediction cast a prediction from getPrediction function '''
''' from a list prediction to a string then dictionnary with json_loads '''
def jsonPrediction(pred):
    string_prediction = str(pred[0])[2:-3] #removing 2 x first and 3 last characters of pred
    json_prediction = json.loads(string_prediction)
    return json_prediction


''' getTrashLabel return label from a frame_to_box'''
def getTrashLabel(frame_2_box):
    return frame_2_box['label']

''' mapLabelTrashId is a switch that converts label to TrashId'''
''' param is label that comes from AI predictions dictionnary jsonPrediction'''
def mapLabel2TrashId(label):
    switcher = { 
    "Fishing or Hunting":"89B44BAA-69AA-4109-891A-128E012E7E07",
    "Food Packaging":"185FEFA2-EEF2-47A8-873E-26032A4BB3C3",
    "Unknown":"BB4DEA69-218A-40CC-A000-2AE17C37152C",
    "Industrial or Construction Debris":"2A863E38-E5D0-455F-87CE-2B75DA29F59A",
    "fragments":"ED401B92-DC24-44C0-A52A-34CE831092BF",
    "Agricultural Waste":"36B2AFEB-7A7C-44B5-A790-5E5C73BA144D",
    "others":"4BEC18FC-BC48-45B7-AFDA-6BA96BD80921",
    "Common Household Items":"C68E90CF-6E65-4474-BC60-72E1C8513F55",
    "plastic":"6961D0DB-928C-419E-9985-98EEEAF552C7",
    "bottles":"9780940B-D06C-4AAB-8003-AB914981E87A",
    "Drinking Bottles":"BCF549A8-AECD-4BC9-B9B8-B94A8F3758D5",
    "Unknown10":"BC7BB564-BE04-4B4B-9913-FF69780B93A6"
    } 
    return switcher.get(label, "nothing")

## GPS

In [ ]:
# Parse GPX file
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm

def goproToGPX(video_name):
    gopro2gpx_script = ['./gopro2gpx_param.sh',video_name]
    result = subprocess.Popen(gopro2gpx_script, stdout=subprocess.PIPE)
    output = []
    i = 0
    for line in result.stdout:
        print(line)
        output.append(line)
    path='/tmp/'+video_name+'.gpx'
    return path

# Create GPX points list
''' gpsPointList function extract gps points from gpx file '''
''' gpxdata is a gpxpy object that returns data from a parsed gpx file'''
''' gpsPointList return a list of dictionnary points with Time, Lat, Long, Elev'''
def gpsPointList(gpxdata):
    point_list = []
    for track in gpxdata.tracks:
        for segment in track.segments: 
            for point in segment.points:
                point_info = {'Time':point.time,'Latitude':point.latitude,'Longitude':point.longitude,'Elevation':point.elevation}
                point_list.append(point_info)
    return point_list


#===============================
def getMediaInfo(mediafile):
    cmd = "mediainfo --Output=JSON %s"%(mediafile)
    proc = subprocess.Popen(cmd, shell=True,stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, stderr = proc.communicate()
    data = json.loads(stdout)
    return data

#===============================
def getDuration(mediafile):
    data = getMediaInfo(mediafile)
    duration = float(data['media']['track'][0]['Duration'])
    return duration


def createTime(time):
    new_time = time
    new_time = new_time + timedelta(seconds=1)
    return new_time

def createLatitude(lat1,lat2):
    new_latitude = (lat1+lat2)/2
    new_latitude = round(new_latitude,6)
    return new_latitude

def createLongitude(long1,long2):
    new_longitude = (long1+long2)/2
    new_latitude = round(new_longitude,6)
    return new_longitude

def createElevation(elev1,elev2):
    new_elevation = (elev1+elev2)/2
    new_elevation = round(new_elevation,6)
    return new_elevation


def fillGPS(inputGPSList,videoLength):
    filledGps = inputGPSList.copy()
    gps_length = len(filledGps)
    iteration_length = int((filledGps[gps_length-1]['Time'] - filledGps[0]['Time']).total_seconds())
    #print(iteration_length)
    ## this section output a filled gps list of length iteration_length+1 = Delta T between last gps timestamp and first one
    i = 0
    while i < (iteration_length):
#        print(i)
        delta = filledGps[i+1]['Time']-filledGps[i]['Time']
        delta = int(delta.total_seconds())
        if delta > 1: # adding a newly created element at index i+1
            #print(i)
            missing_time = createTime(filledGps[i]['Time'])
            missing_latitude = createLatitude(filledGps[i]['Latitude'],filledGps[i+1]['Latitude'])
            missing_longitude = createLongitude(filledGps[i]['Longitude'],filledGps[i+1]['Longitude'])
            missing_elevation = createElevation(filledGps[i]['Elevation'],filledGps[i+1]['Elevation'])
            new_gps = {'Time':missing_time,'Latitude':missing_latitude,'Longitude':missing_longitude,'Elevation':missing_elevation}
            filledGps.insert(i+1,new_gps)
        i = i+1
    ## this section add missing point at the end of the list, in case filledGps initial Delta time length is less than actual video length
    if len(filledGps) < videoLength:
        j = 0
        while len(filledGps) < videoLength:
            filledGps.insert(len(filledGps),filledGps[len(filledGps)-1])
            j = j+1

    return filledGps


def longLat2shapePoint(gpsLongLatPoint):
    gpsShapePoint = {'Time':gpsLongLatPoint['Time'],'the_geom':Point(gpsLongLatPoint['Longitude'],gpsLongLatPoint['Latitude']),'Elevation':gpsLongLatPoint['Elevation']}
    return gpsShapePoint

def longLat2shapeList(gpsLongLagList):
    gpsShapeList = []
    for gpsPoint in gpsLongLagList:
        gpsShapePoint = longLat2shapePoint(gpsPoint)
        gpsShapeList.append(gpsShapePoint)
    return gpsShapeList



def geometryTransfo(gpsShapePoint):
    project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:2154')) # destination coordinate system

    geo1 = gpsShapePoint['the_geom']
    geo2 = transform(project,geo1)

    return geo2

def gps2154(gpsShapePointsFilled):
    gps2154Points = []
    for point in tqdm(gpsShapePointsFilled):
        geo2154 = geometryTransfo(point)
        gps2154Point = {'Time':point['Time'],'the_geom':geo2154,'Elevation':point['Elevation']}
        gps2154Points.append(gps2154Point)
    return gps2154Points

## PostGre

In [ ]:
import psycopg2

def pgOpenConnection(conn_string):
    try:
        conn = psycopg2.connect(conn_string)
        print("Connection established")
        return conn
    except psycopg2.OperationalError as err:
        print("Connection could not established: ",err)
    #return conn

# Function to close connection to PG server
def pgCloseConnection(connection):
    try:
        connection.close()
        print("PG connection closed")
    except:
        print("PG connection could not close successfully")

# Label to TrashId PGSQL Data Model
def mapLabel2TrashIdPG(label):
    switcher = { 
        "others":"1", #"autre dechet" in PG Data Model mapped to IA "others" label
        "dechet agricole":"2",
        "bottles":"3", #"bouteille boisson" in PG Data Model mapped to IA "bottles" label
        "fragments":"4",#"industriel ou construction in PG Data Model mapped to IA "fragments" label
        "peche et chasse":"5",
        "emballage alimentaire":"6",
        "objet vie courante":"7",
        "autres dechets +10":"8"
    } 
    return switcher.get(label, "nothing")

# Dummy Function to give GPS point to Detected Trash, based on TrashId
def trashGPS(trashId,gps2154Points):
    length = len(gps2154Points)+1
    gpsIndex = trashId % length
    return gpsIndex


# INSERT REQUEST: point parameter + trashTypeId + Timestamp
def trashInsert(gps2154Point,trashTypeId,cursor,connexion):
    point = 'POINT(' + str(gps2154Point['the_geom'].coords.xy[0][0]) + ' ' + str(gps2154Point['the_geom'].coords.xy[1][0]) + ')'
    elevation = gps2154Point['Elevation']
    timestamp = gps2154Point['Time']
    cursor.execute("INSERT INTO public.trash (id, id_ref_campaign_fk,the_geom, elevation, id_ref_trash_type_fk,brand_type,time ) VALUES (DEFAULT, '003d8675-29e3-4cde-a315-095ac2ec80bc',%s,%s,%s,%s,%s) RETURNING id;", (point,elevation,trashTypeId,'icetea',timestamp))
    connexion.commit()
    row_id = cursor.fetchone()[0]
    return row_id

## Main

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def main():

    ######## Pipeline Step0: Get Video to predict and insert#########
    print('######## Pipeline Step0: Get Video from Azure Blob Storage #########')
    # blob storage connection string
    connection_string = os.getenv("CONN_STRING")

    # get list of blobs in container campaign0
    campaign_container_name = 'campaign0'
    blobs_campaign0 = blobInContainer(connection_string,campaign_container_name)
    print(blobs_campaign0)

    # get infos of blob 'goproshort-480p.mov' 
    blob_video_name = 'goproshort-480p.mov'   
    blobInfos(connection_string,campaign_container_name,blob_video_name)

    # download locally in /tmp blob video
    blob_video0 = BlobClient.from_connection_string(conn_str=connection_string,container_name=campaign_container_name, blob_name=blob_video_name)
    downloadBlob(blob_video0)

    ######## Pipeline Step 1bis: AI Trash prediction #########
    print('######## Pipeline Step 1bis: AI Trash prediction #########')
    isAIready('http://aiapisurfrider.northeurope.cloudapp.azure.com:5000')
    # get predictions from AI on goproshort-480p.mov
    prediction = getPrediction(blob_video_name)

    # cast prediction to JSON/Dictionnary format
    json_prediction = jsonPrediction(prediction)
    json_prediction

    ######## Pipeline Step 1: GPX creation ########
    print('######## Pipeline Step 1: GPX creation ########')
    video_name = '28022020_Boudigau_4.MP4'
    gpx_path = goproToGPX(video_name)
    gpx_path

    # GPX parsing
    gpx_file = open(gpx_path,'r',encoding='utf-8')
    gpx_data = gpxpy.parse(gpx_file) # data from parsed gpx file

    # GPS Points
    gpsPoints = gpsPointList(gpx_data)

    # Video duration
    print("\n")
    video_duration = getDuration('/tmp/'+video_name)
    print("Video duration in second from metadata:",video_duration)

    # GPS file duration
    timestampDelta = gpsPoints[len(gpsPoints)-1]['Time'] - gpsPoints[0]['Time']
    print("GPS file time coverage in second: ",timestampDelta.seconds)

    ######## Pipeline Step 2: Create gpsPointFilled ########
    print('######## Pipeline Step 2: Create gpsPointFilled ########')
    video_duration_sup = int(video_duration)+1
    gpsPointsFilled = fillGPS(gpsPoints,video_duration_sup)

    ######## Pipeline Step 3: GPS shapePoints ########
    print('######## Pipeline Step 3: GPS shapePoints ########')
    gpsShapePointsFilled = longLat2shapeList(gpsPointsFilled)

    ######## Pipeline Step 4: 2154 geometry conversion ########
    print('######## Pipeline Step 4: 2154 Geometry conversion ########')
    gps2154PointsFilled = gps2154(gpsShapePointsFilled)

    ######## Pipeline Step 5: PostGre INSERT ########
    print('######## Pipeline Step 5: PostGre INSERT ########')
    # Postgre connection pre-requesite
    # Update connection string information from env variables
    pgserver = os.getenv("PGSERVER")
    pgdatabase = os.getenv("PGDATABASE")
    pgusername = os.getenv("PGUSERNAME")
    pgpassword = os.getenv("PGPWD")
    sslmode = "require"

    # Open pgConnection, create cursor
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(pgserver, pgusername, pgdatabase, pgpassword, sslmode)
    pgconnection = pgOpenConnection(conn_string)
    cursor = pgconnection.cursor()


    # INSERTING all detected_trash within PostGre
    rowID_list = []
    for prediction in tqdm(json_prediction['detected_trash']):
        try: 
            # get GPS coordinate
            trashTypeId= prediction['id']
            gpsIndexId = trashGPS(trashTypeId,gps2154PointsFilled)
            gpsTrashTypeId = gps2154PointsFilled[gpsIndexId]
            # get TrashTypeId from AI prediction
            label = getTrashLabel(prediction)
            trashType = mapLabel2TrashIdPG(label)
            # INSERT within PostGRE
            rowID = trashInsert(gpsTrashTypeId,trashType,cursor,pgconnection)
            print("prediction:",prediction['id'])
            print("rowID:",rowID)
            rowID_list.append(rowID)
        except:
            print("There was an issue inserting Trash id:" + str(prediction['id']) + " within PostGre")
    print("Successfully inserted " + str(len(rowID_list)) + " Trashes within Trash table")    

    # Close PG connection
    pgCloseConnection(pgconnection)

In [ ]:
# Execute main function
if __name__ == '__main__':
    main()